In [1]:
import os
import gc
import time
import numpy as np
import pandas as pd
from contextlib import contextmanager
import multiprocessing as mp
from functools import partial
from scipy.stats import kurtosis, iqr, skew
from lightgbm import LGBMClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def one_hot_encoder(df, categorical_columns=None, nan_as_category=True):
    """Create a new column for each categorical value in categorical columns. """
    original_columns = list(df.columns)
    if not categorical_columns:
        categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    categorical_columns = [c for c in df.columns if c not in original_columns]
    return df, categorical_columns

In [3]:
def group(df_to_agg, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    agg_df = df_to_agg.groupby(aggregate_by).agg(aggregations)
    agg_df.columns = pd.Index(['{}{}_{}'.format(prefix, e[0], e[1].upper())
                               for e in agg_df.columns.tolist()])
    return agg_df.reset_index()


In [4]:
def group_and_merge(df_to_agg, df_to_merge, prefix, aggregations, aggregate_by= 'SK_ID_CURR'):
    agg_df = group(df_to_agg, prefix, aggregations, aggregate_by= aggregate_by)
    return df_to_merge.merge(agg_df, how='left', on= aggregate_by)

# BUREAU PIPELINE

In [5]:
def get_bureau_balance(path, num_rows= None):
    bb = pd.read_csv(os.path.join(path, 'bureau_balance.csv'), nrows= num_rows)
    bb, categorical_cols = one_hot_encoder(bb, nan_as_category= False)
    # Calculate rate for each category with decay
    bb_processed = bb.groupby('SK_ID_BUREAU')[categorical_cols].mean().reset_index()
    # Min, Max, Count and mean duration of payments (months)
    agg = {'MONTHS_BALANCE': ['min', 'max', 'mean', 'size']}
    bb_processed = group_and_merge(bb, bb_processed, '', agg, 'SK_ID_BUREAU')
    del bb; gc.collect()
    return bb_processed

In [7]:
""" Process bureau.csv and bureau_balance.csv and return a pandas dataframe. """
bureau = pd.read_csv('../input/bureau.csv', nrows= None)
# Credit duration and credit/account end date difference
bureau['CREDIT_DURATION'] = -bureau['DAYS_CREDIT'] + bureau['DAYS_CREDIT_ENDDATE']
bureau['ENDDATE_DIF'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_ENDDATE_FACT']
# Credit to debt ratio and difference
bureau['DEBT_PERCENTAGE'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_CREDIT_SUM_DEBT']
bureau['DEBT_CREDIT_DIFF'] = bureau['AMT_CREDIT_SUM'] - bureau['AMT_CREDIT_SUM_DEBT']
bureau['CREDIT_TO_ANNUITY_RATIO'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_ANNUITY']



In [5]:
bureau = pd.read_csv('../input/bureau.csv', nrows= None)
bureau.head(5)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [8]:
# One-hot encoder
bureau, categorical_cols = one_hot_encoder(bureau, nan_as_category= False)


In [10]:
bb = pd.read_csv('../input/bureau_balance.csv', nrows= None)
bb, categorical_cols = one_hot_encoder(bb, nan_as_category= False)
# Calculate rate for each category with decay
bb_processed = bb.groupby('SK_ID_BUREAU')[categorical_cols].mean().reset_index()
# Min, Max, Count and mean duration of payments (months)
agg = {'MONTHS_BALANCE': ['min', 'max', 'mean', 'size']}
bb_processed = group_and_merge(bb, bb_processed, '', agg, 'SK_ID_BUREAU')
del bb; gc.collect()

147

In [11]:
bureau = bureau.merge(bb_processed, how='left', on='SK_ID_BUREAU')

In [13]:
# Flag months with late payments (days past due)
bureau['STATUS_12345'] = 0
for i in range(1,6):
    bureau['STATUS_12345'] += bureau['STATUS_{}'.format(i)]

In [14]:
# Aggregate by number of months in balance and merge with bureau (loan length agg)
features = ['AMT_CREDIT_MAX_OVERDUE', 'AMT_CREDIT_SUM_OVERDUE', 'AMT_CREDIT_SUM',
    'AMT_CREDIT_SUM_DEBT', 'DEBT_PERCENTAGE', 'DEBT_CREDIT_DIFF', 'STATUS_0', 'STATUS_12345']
agg_length = bureau.groupby('MONTHS_BALANCE_SIZE')[features].mean().reset_index()

In [15]:
agg_length.head(5)

,MONTHS_BALANCE_SIZE,AMT_CREDIT_MAX_OVERDUE,AMT_CREDIT_SUM_OVERDUE,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,DEBT_PERCENTAGE,DEBT_CREDIT_DIFF,STATUS_0,STATUS_12345
0,1.0,323.042068,7.265145,204378.412977,153632.203666,inf,33327.636173,0.689702,0.001355
1,2.0,908.651492,19.100507,220169.754970,144735.901552,inf,61775.723039,0.569105,0.004829
2,3.0,917.692412,8.348935,284455.912587,192833.202089,inf,71513.378535,0.595704,0.009763
3,4.0,1184.503495,21.166049,339286.264661,225915.137663,inf,97843.348817,0.640054,0.012940
4,5.0,1585.077346,36.463318,381171.978068,244024.880755,inf,113672.517764,0.655135,0.016202


In [16]:
agg_length.rename({feat: 'LL_' + feat for feat in features}, axis=1, inplace=True)
bureau = bureau.merge(agg_length, how='left', on='MONTHS_BALANCE_SIZE')

In [17]:
bureau.head(5)

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,...,MONTHS_BALANCE_SIZE,STATUS_12345,LL_AMT_CREDIT_MAX_OVERDUE,LL_AMT_CREDIT_SUM_OVERDUE,LL_AMT_CREDIT_SUM,LL_AMT_CREDIT_SUM_DEBT,LL_DEBT_PERCENTAGE,LL_DEBT_CREDIT_DIFF,LL_STATUS_0,LL_STATUS_12345
0,215354,5714462,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,215354,5714463,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,215354,5714464,-203,0,528.0,NaN,NaN,0,464323.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,215354,5714465,-203,0,NaN,NaN,NaN,0,90000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,215354,5714466,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
BUREAU_AGG = {
    'SK_ID_BUREAU': ['nunique'],
    'DAYS_CREDIT': ['min', 'max', 'mean'],
    'DAYS_CREDIT_ENDDATE': ['min', 'max'],
    'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_OVERDUE': ['max', 'mean', 'sum'],
    'AMT_ANNUITY': ['mean'],
    'DEBT_CREDIT_DIFF': ['mean', 'sum'],
    'MONTHS_BALANCE_MEAN': ['mean', 'var'],
    'MONTHS_BALANCE_SIZE': ['mean', 'sum'],
    # Categorical
    'STATUS_0': ['mean'],
    'STATUS_1': ['mean'],
    'STATUS_12345': ['mean'],
    'STATUS_C': ['mean'],
    'STATUS_X': ['mean'],
    'CREDIT_ACTIVE_Active': ['mean'],
    'CREDIT_ACTIVE_Closed': ['mean'],
    'CREDIT_ACTIVE_Sold': ['mean'],
    'CREDIT_TYPE_Consumer credit': ['mean'],
    'CREDIT_TYPE_Credit card': ['mean'],
    'CREDIT_TYPE_Car loan': ['mean'],
    'CREDIT_TYPE_Mortgage': ['mean'],
    'CREDIT_TYPE_Microloan': ['mean'],
    # Group by loan duration features (months)
    'LL_AMT_CREDIT_SUM_OVERDUE': ['mean'],
    'LL_DEBT_CREDIT_DIFF': ['mean'],
    'LL_STATUS_12345': ['mean'],
}

In [21]:
# General loans aggregations
agg_bureau = group(bureau, 'BUREAU_', BUREAU_AGG)

In [22]:
agg_bureau.head(5)

,SK_ID_CURR,BUREAU_SK_ID_BUREAU_NUNIQUE,BUREAU_DAYS_CREDIT_MIN,BUREAU_DAYS_CREDIT_MAX,BUREAU_DAYS_CREDIT_MEAN,BUREAU_DAYS_CREDIT_ENDDATE_MIN,BUREAU_DAYS_CREDIT_ENDDATE_MAX,BUREAU_AMT_CREDIT_MAX_OVERDUE_MAX,BUREAU_AMT_CREDIT_MAX_OVERDUE_MEAN,BUREAU_AMT_CREDIT_SUM_MAX,...,BUREAU_CREDIT_ACTIVE_Closed_MEAN,BUREAU_CREDIT_ACTIVE_Sold_MEAN,BUREAU_CREDIT_TYPE_Consumer credit_MEAN,BUREAU_CREDIT_TYPE_Credit card_MEAN,BUREAU_CREDIT_TYPE_Car loan_MEAN,BUREAU_CREDIT_TYPE_Mortgage_MEAN,BUREAU_CREDIT_TYPE_Microloan_MEAN,BUREAU_LL_AMT_CREDIT_SUM_OVERDUE_MEAN,BUREAU_LL_DEBT_CREDIT_DIFF_MEAN,BUREAU_LL_STATUS_12345_MEAN
0,100001,7,-1572,-49,-735.000000,-1329.0,1778.0,NaN,NaN,378000.0,...,0.571429,0.0,1.000000,0.000000,0.0,0.0,0.0,23.310086,198425.232046,0.012390
1,100002,8,-1437,-103,-874.000000,-1072.0,780.0,5043.645,1681.029,450000.0,...,0.750000,0.0,0.500000,0.500000,0.0,0.0,0.0,78.249173,179454.781051,0.016089
2,100003,4,-2586,-606,-1400.750000,-2434.0,1216.0,0.000,0.000,810000.0,...,0.750000,0.0,0.500000,0.500000,0.0,0.0,0.0,NaN,NaN,NaN
3,100004,2,-1326,-408,-867.000000,-595.0,-382.0,0.000,0.000,94537.8,...,1.000000,0.0,1.000000,0.000000,0.0,0.0,0.0,NaN,NaN,NaN
4,100005,3,-373,-62,-190.666667,-128.0,1324.0,0.000,0.000,568800.0,...,0.333333,0.0,0.666667,0.333333,0.0,0.0,0.0,31.024979,123475.451593,0.014672


In [23]:
# Active and closed loans aggregations
active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
active.head(5)

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,...,MONTHS_BALANCE_SIZE,STATUS_12345,LL_AMT_CREDIT_MAX_OVERDUE,LL_AMT_CREDIT_SUM_OVERDUE,LL_AMT_CREDIT_SUM,LL_AMT_CREDIT_SUM_DEBT,LL_DEBT_PERCENTAGE,LL_DEBT_CREDIT_DIFF,LL_STATUS_0,LL_STATUS_12345
1,215354,5714463,-208,0,1075.0,NaN,NaN,0,225000.0,171342.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,215354,5714464,-203,0,528.0,NaN,NaN,0,464323.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,215354,5714465,-203,0,NaN,NaN,NaN,0,90000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,215354,5714466,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,215354,5714467,-273,0,27460.0,NaN,0.0,0,180000.0,71017.38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
list(active)

['SK_ID_CURR',
 'SK_ID_BUREAU',
 'DAYS_CREDIT',
 'CREDIT_DAY_OVERDUE',
 'DAYS_CREDIT_ENDDATE',
 'DAYS_ENDDATE_FACT',
 'AMT_CREDIT_MAX_OVERDUE',
 'CNT_CREDIT_PROLONG',
 'AMT_CREDIT_SUM',
 'AMT_CREDIT_SUM_DEBT',
 'AMT_CREDIT_SUM_LIMIT',
 'AMT_CREDIT_SUM_OVERDUE',
 'DAYS_CREDIT_UPDATE',
 'AMT_ANNUITY',
 'CREDIT_DURATION',
 'ENDDATE_DIF',
 'DEBT_PERCENTAGE',
 'DEBT_CREDIT_DIFF',
 'CREDIT_TO_ANNUITY_RATIO',
 'CREDIT_ACTIVE_Active',
 'CREDIT_ACTIVE_Bad debt',
 'CREDIT_ACTIVE_Closed',
 'CREDIT_ACTIVE_Sold',
 'CREDIT_CURRENCY_currency 1',
 'CREDIT_CURRENCY_currency 2',
 'CREDIT_CURRENCY_currency 3',
 'CREDIT_CURRENCY_currency 4',
 'CREDIT_TYPE_Another type of loan',
 'CREDIT_TYPE_Car loan',
 'CREDIT_TYPE_Cash loan (non-earmarked)',
 'CREDIT_TYPE_Consumer credit',
 'CREDIT_TYPE_Credit card',
 'CREDIT_TYPE_Interbank credit',
 'CREDIT_TYPE_Loan for business development',
 'CREDIT_TYPE_Loan for purchase of shares (margin lending)',
 'CREDIT_TYPE_Loan for the purchase of equipment',
 'CREDIT_TYPE_L

In [26]:
BUREAU_ACTIVE_AGG = {
    'DAYS_CREDIT': ['max', 'mean'],
    'DAYS_CREDIT_ENDDATE': ['min', 'max'],
    'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_SUM': ['max', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['mean', 'sum'],
    'AMT_CREDIT_SUM_OVERDUE': ['max', 'mean'],
    'DAYS_CREDIT_UPDATE': ['min', 'mean'],
    'DEBT_PERCENTAGE': ['mean'],
    'DEBT_CREDIT_DIFF': ['mean'],
    'CREDIT_TO_ANNUITY_RATIO': ['mean'],
    'MONTHS_BALANCE_MEAN': ['mean', 'var'],
    'MONTHS_BALANCE_SIZE': ['mean', 'sum'],
}

In [27]:
BUREAU_CLOSED_AGG = {
    'DAYS_CREDIT': ['max', 'var'],
    'DAYS_CREDIT_ENDDATE': ['max'],
    'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_SUM_OVERDUE': ['mean'],
    'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['max', 'sum'],
    'DAYS_CREDIT_UPDATE': ['max'],
    'ENDDATE_DIF': ['mean'],
    'STATUS_12345': ['mean'],
}

In [28]:
agg_bureau = group_and_merge(active,agg_bureau,'BUREAU_ACTIVE_',BUREAU_ACTIVE_AGG)
closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
agg_bureau = group_and_merge(closed,agg_bureau,'BUREAU_CLOSED_',BUREAU_CLOSED_AGG)

In [29]:
agg_bureau.head(5)

,SK_ID_CURR,BUREAU_SK_ID_BUREAU_NUNIQUE,BUREAU_DAYS_CREDIT_MIN,BUREAU_DAYS_CREDIT_MAX,BUREAU_DAYS_CREDIT_MEAN,BUREAU_DAYS_CREDIT_ENDDATE_MIN,BUREAU_DAYS_CREDIT_ENDDATE_MAX,BUREAU_AMT_CREDIT_MAX_OVERDUE_MAX,BUREAU_AMT_CREDIT_MAX_OVERDUE_MEAN,BUREAU_AMT_CREDIT_SUM_MAX,...,BUREAU_CLOSED_AMT_CREDIT_MAX_OVERDUE_MEAN,BUREAU_CLOSED_AMT_CREDIT_SUM_OVERDUE_MEAN,BUREAU_CLOSED_AMT_CREDIT_SUM_MAX,BUREAU_CLOSED_AMT_CREDIT_SUM_MEAN,BUREAU_CLOSED_AMT_CREDIT_SUM_SUM,BUREAU_CLOSED_AMT_CREDIT_SUM_DEBT_MAX,BUREAU_CLOSED_AMT_CREDIT_SUM_DEBT_SUM,BUREAU_CLOSED_DAYS_CREDIT_UPDATE_MAX,BUREAU_CLOSED_ENDDATE_DIF_MEAN,BUREAU_CLOSED_STATUS_12345_MEAN
0,100001,7,-1572,-49,-735.000000,-1329.0,1778.0,NaN,NaN,378000.0,...,NaN,0.0,279720.0,142335.0,569340.0,0.0,0.0,-155.0,197.0,0.000000
1,100002,8,-1437,-103,-874.000000,-1072.0,780.0,5043.645,1681.029,450000.0,...,2091.16125,0.0,135000.0,63844.5,383067.0,0.0,0.0,-34.0,252.6,0.278409
2,100003,4,-2586,-606,-1400.750000,-2434.0,1216.0,0.000,0.000,810000.0,...,0.00000,0.0,112500.0,69133.5,207400.5,0.0,0.0,-540.0,-34.0,NaN
3,100004,2,-1326,-408,-867.000000,-595.0,-382.0,0.000,0.000,94537.8,...,0.00000,0.0,94537.8,94518.9,189037.8,0.0,0.0,-382.0,44.0,NaN
4,100005,3,-373,-62,-190.666667,-128.0,1324.0,0.000,0.000,568800.0,...,NaN,0.0,58500.0,58500.0,58500.0,0.0,0.0,-121.0,-5.0,0.000000


In [30]:
list(bureau)

['SK_ID_CURR',
 'SK_ID_BUREAU',
 'DAYS_CREDIT',
 'CREDIT_DAY_OVERDUE',
 'DAYS_CREDIT_ENDDATE',
 'DAYS_ENDDATE_FACT',
 'AMT_CREDIT_MAX_OVERDUE',
 'CNT_CREDIT_PROLONG',
 'AMT_CREDIT_SUM',
 'AMT_CREDIT_SUM_DEBT',
 'AMT_CREDIT_SUM_LIMIT',
 'AMT_CREDIT_SUM_OVERDUE',
 'DAYS_CREDIT_UPDATE',
 'AMT_ANNUITY',
 'CREDIT_DURATION',
 'ENDDATE_DIF',
 'DEBT_PERCENTAGE',
 'DEBT_CREDIT_DIFF',
 'CREDIT_TO_ANNUITY_RATIO',
 'CREDIT_ACTIVE_Active',
 'CREDIT_ACTIVE_Bad debt',
 'CREDIT_ACTIVE_Closed',
 'CREDIT_ACTIVE_Sold',
 'CREDIT_CURRENCY_currency 1',
 'CREDIT_CURRENCY_currency 2',
 'CREDIT_CURRENCY_currency 3',
 'CREDIT_CURRENCY_currency 4',
 'CREDIT_TYPE_Another type of loan',
 'CREDIT_TYPE_Car loan',
 'CREDIT_TYPE_Cash loan (non-earmarked)',
 'CREDIT_TYPE_Consumer credit',
 'CREDIT_TYPE_Credit card',
 'CREDIT_TYPE_Interbank credit',
 'CREDIT_TYPE_Loan for business development',
 'CREDIT_TYPE_Loan for purchase of shares (margin lending)',
 'CREDIT_TYPE_Loan for the purchase of equipment',
 'CREDIT_TYPE_L

In [32]:
BUREAU_LOAN_TYPE_AGG = {
    'DAYS_CREDIT': ['mean', 'max'],
    'AMT_CREDIT_MAX_OVERDUE': ['mean', 'max'],
    'AMT_CREDIT_SUM': ['mean', 'max'],
    'AMT_CREDIT_SUM_DEBT': ['mean', 'max'],
    'DEBT_PERCENTAGE': ['mean'],
    'DEBT_CREDIT_DIFF': ['mean'],
    'DAYS_CREDIT_ENDDATE': ['max'],
}

In [33]:
# Aggregations for the main loan types
for credit_type in ['Consumer credit', 'Credit card', 'Mortgage', 'Car loan', 'Microloan']:
    type_df = bureau[bureau['CREDIT_TYPE_' + credit_type] == 1]
    prefix = 'BUREAU_' + credit_type.split(' ')[0].upper() + '_'
    agg_bureau = group_and_merge(type_df, agg_bureau, prefix, BUREAU_LOAN_TYPE_AGG)
    del type_df; gc.collect()
agg_bureau.head(5)

,SK_ID_CURR,BUREAU_SK_ID_BUREAU_NUNIQUE,BUREAU_DAYS_CREDIT_MIN,BUREAU_DAYS_CREDIT_MAX,BUREAU_DAYS_CREDIT_MEAN,BUREAU_DAYS_CREDIT_ENDDATE_MIN,BUREAU_DAYS_CREDIT_ENDDATE_MAX,BUREAU_AMT_CREDIT_MAX_OVERDUE_MAX,BUREAU_AMT_CREDIT_MAX_OVERDUE_MEAN,BUREAU_AMT_CREDIT_SUM_MAX,...,BUREAU_MICROLOAN_DAYS_CREDIT_MAX,BUREAU_MICROLOAN_AMT_CREDIT_MAX_OVERDUE_MEAN,BUREAU_MICROLOAN_AMT_CREDIT_MAX_OVERDUE_MAX,BUREAU_MICROLOAN_AMT_CREDIT_SUM_MEAN,BUREAU_MICROLOAN_AMT_CREDIT_SUM_MAX,BUREAU_MICROLOAN_AMT_CREDIT_SUM_DEBT_MEAN,BUREAU_MICROLOAN_AMT_CREDIT_SUM_DEBT_MAX,BUREAU_MICROLOAN_DEBT_PERCENTAGE_MEAN,BUREAU_MICROLOAN_DEBT_CREDIT_DIFF_MEAN,BUREAU_MICROLOAN_DAYS_CREDIT_ENDDATE_MAX
0,100001,7,-1572,-49,-735.000000,-1329.0,1778.0,NaN,NaN,378000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100002,8,-1437,-103,-874.000000,-1072.0,780.0,5043.645,1681.029,450000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100003,4,-2586,-606,-1400.750000,-2434.0,1216.0,0.000,0.000,810000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100004,2,-1326,-408,-867.000000,-595.0,-382.0,0.000,0.000,94537.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100005,3,-373,-62,-190.666667,-128.0,1324.0,0.000,0.000,568800.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
BUREAU_TIME_AGG = {
    'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean'],
    'AMT_CREDIT_SUM_OVERDUE': ['mean'],
    'AMT_CREDIT_SUM': ['max', 'sum'],
    'AMT_CREDIT_SUM_DEBT': ['mean', 'sum'],
    'DEBT_PERCENTAGE': ['mean'],
    'DEBT_CREDIT_DIFF': ['mean'],
    'STATUS_0': ['mean'],
    'STATUS_12345': ['mean'],
}

In [40]:
for time_frame in [6, 12]:
    prefix = "BUREAU_LAST{}M_".format(time_frame)
    time_frame_df = bureau[bureau['DAYS_CREDIT'] >= -30*time_frame]
    agg_bureau = group_and_merge(time_frame_df, agg_bureau, prefix, BUREAU_TIME_AGG)
    del time_frame_df; gc.collect()
agg_bureau.head(5)

,SK_ID_CURR,BUREAU_SK_ID_BUREAU_NUNIQUE,BUREAU_DAYS_CREDIT_MIN,BUREAU_DAYS_CREDIT_MAX,BUREAU_DAYS_CREDIT_MEAN,BUREAU_DAYS_CREDIT_ENDDATE_MIN,BUREAU_DAYS_CREDIT_ENDDATE_MAX,BUREAU_AMT_CREDIT_MAX_OVERDUE_MAX,BUREAU_AMT_CREDIT_MAX_OVERDUE_MEAN,BUREAU_AMT_CREDIT_SUM_MAX,...,BUREAU_LAST12M_AMT_CREDIT_MAX_OVERDUE_MEAN,BUREAU_LAST12M_AMT_CREDIT_SUM_OVERDUE_MEAN,BUREAU_LAST12M_AMT_CREDIT_SUM_MAX,BUREAU_LAST12M_AMT_CREDIT_SUM_SUM,BUREAU_LAST12M_AMT_CREDIT_SUM_DEBT_MEAN,BUREAU_LAST12M_AMT_CREDIT_SUM_DEBT_SUM,BUREAU_LAST12M_DEBT_PERCENTAGE_MEAN,BUREAU_LAST12M_DEBT_CREDIT_DIFF_MEAN,BUREAU_LAST12M_STATUS_0_MEAN,BUREAU_LAST12M_STATUS_12345_MEAN
0,100001,7,-1572,-49,-735.000000,-1329.0,1778.0,NaN,NaN,378000.0,...,NaN,0.0,378000.000,546345.000,241760.25,483520.5,1.269629,31412.250,0.750000,0.0
1,100002,8,-1437,-103,-874.000000,-1072.0,780.0,5043.645,1681.029,450000.0,...,40.5,0.0,31988.565,31988.565,0.00,0.0,inf,31988.565,0.500000,0.0
2,100003,4,-2586,-606,-1400.750000,-2434.0,1216.0,0.000,0.000,810000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100004,2,-1326,-408,-867.000000,-595.0,-382.0,0.000,0.000,94537.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100005,3,-373,-62,-190.666667,-128.0,1324.0,0.000,0.000,568800.0,...,0.0,0.0,568800.000,598626.000,284204.25,568408.5,1.112619,15108.750,0.833333,0.0


In [41]:
# Last loan max overdue
sort_bureau = bureau.sort_values(by=['DAYS_CREDIT'])
gr = sort_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_MAX_OVERDUE'].last().reset_index()
gr.rename({'AMT_CREDIT_MAX_OVERDUE': 'BUREAU_LAST_LOAN_MAX_OVERDUE'}, inplace=True)
agg_bureau = agg_bureau.merge(gr, on='SK_ID_CURR', how='left')

In [42]:
agg_bureau.head(5)

,SK_ID_CURR,BUREAU_SK_ID_BUREAU_NUNIQUE,BUREAU_DAYS_CREDIT_MIN,BUREAU_DAYS_CREDIT_MAX,BUREAU_DAYS_CREDIT_MEAN,BUREAU_DAYS_CREDIT_ENDDATE_MIN,BUREAU_DAYS_CREDIT_ENDDATE_MAX,BUREAU_AMT_CREDIT_MAX_OVERDUE_MAX,BUREAU_AMT_CREDIT_MAX_OVERDUE_MEAN,BUREAU_AMT_CREDIT_SUM_MAX,...,BUREAU_LAST12M_AMT_CREDIT_SUM_OVERDUE_MEAN,BUREAU_LAST12M_AMT_CREDIT_SUM_MAX,BUREAU_LAST12M_AMT_CREDIT_SUM_SUM,BUREAU_LAST12M_AMT_CREDIT_SUM_DEBT_MEAN,BUREAU_LAST12M_AMT_CREDIT_SUM_DEBT_SUM,BUREAU_LAST12M_DEBT_PERCENTAGE_MEAN,BUREAU_LAST12M_DEBT_CREDIT_DIFF_MEAN,BUREAU_LAST12M_STATUS_0_MEAN,BUREAU_LAST12M_STATUS_12345_MEAN,AMT_CREDIT_MAX_OVERDUE
0,100001,7,-1572,-49,-735.000000,-1329.0,1778.0,NaN,NaN,378000.0,...,0.0,378000.000,546345.000,241760.25,483520.5,1.269629,31412.250,0.750000,0.0,NaN
1,100002,8,-1437,-103,-874.000000,-1072.0,780.0,5043.645,1681.029,450000.0,...,0.0,31988.565,31988.565,0.00,0.0,inf,31988.565,0.500000,0.0,40.5
2,100003,4,-2586,-606,-1400.750000,-2434.0,1216.0,0.000,0.000,810000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,100004,2,-1326,-408,-867.000000,-595.0,-382.0,0.000,0.000,94537.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,100005,3,-373,-62,-190.666667,-128.0,1324.0,0.000,0.000,568800.0,...,0.0,568800.000,598626.000,284204.25,568408.5,1.112619,15108.750,0.833333,0.0,0.0


In [43]:
# Ratios: total debt/total credit and active loans debt/ active loans credit
agg_bureau['BUREAU_DEBT_OVER_CREDIT'] = \
    agg_bureau['BUREAU_AMT_CREDIT_SUM_DEBT_SUM']/agg_bureau['BUREAU_AMT_CREDIT_SUM_SUM']
agg_bureau['BUREAU_ACTIVE_DEBT_OVER_CREDIT'] = \
    agg_bureau['BUREAU_ACTIVE_AMT_CREDIT_SUM_DEBT_SUM']/agg_bureau['BUREAU_ACTIVE_AMT_CREDIT_SUM_SUM']

In [44]:
agg_bureau.head(5)

,SK_ID_CURR,BUREAU_SK_ID_BUREAU_NUNIQUE,BUREAU_DAYS_CREDIT_MIN,BUREAU_DAYS_CREDIT_MAX,BUREAU_DAYS_CREDIT_MEAN,BUREAU_DAYS_CREDIT_ENDDATE_MIN,BUREAU_DAYS_CREDIT_ENDDATE_MAX,BUREAU_AMT_CREDIT_MAX_OVERDUE_MAX,BUREAU_AMT_CREDIT_MAX_OVERDUE_MEAN,BUREAU_AMT_CREDIT_SUM_MAX,...,BUREAU_LAST12M_AMT_CREDIT_SUM_SUM,BUREAU_LAST12M_AMT_CREDIT_SUM_DEBT_MEAN,BUREAU_LAST12M_AMT_CREDIT_SUM_DEBT_SUM,BUREAU_LAST12M_DEBT_PERCENTAGE_MEAN,BUREAU_LAST12M_DEBT_CREDIT_DIFF_MEAN,BUREAU_LAST12M_STATUS_0_MEAN,BUREAU_LAST12M_STATUS_12345_MEAN,AMT_CREDIT_MAX_OVERDUE,BUREAU_DEBT_OVER_CREDIT,BUREAU_ACTIVE_DEBT_OVER_CREDIT
0,100001,7,-1572,-49,-735.000000,-1329.0,1778.0,NaN,NaN,378000.0,...,546345.000,241760.25,483520.5,1.269629,31412.250,0.750000,0.0,NaN,0.410555,0.674966
1,100002,8,-1437,-103,-874.000000,-1072.0,780.0,5043.645,1681.029,450000.0,...,31988.565,0.00,0.0,inf,31988.565,0.500000,0.0,40.5,0.284122,0.509931
2,100003,4,-2586,-606,-1400.750000,-2434.0,1216.0,0.000,0.000,810000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,0.000000
3,100004,2,-1326,-408,-867.000000,-595.0,-382.0,0.000,0.000,94537.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,NaN
4,100005,3,-373,-62,-190.666667,-128.0,1324.0,0.000,0.000,568800.0,...,598626.000,284204.25,568408.5,1.112619,15108.750,0.833333,0.0,0.0,0.864992,0.949522


# POS-CASH PIPELINE 

In [7]:
pos = pd.read_csv('../input/POS_CASH_balance.csv', nrows= None)
pos, categorical_cols = one_hot_encoder(pos, nan_as_category= False)

In [8]:
POS_CASH_AGG = {
    'SK_ID_PREV': ['nunique'],
    'MONTHS_BALANCE': ['min', 'max', 'size'],
    'SK_DPD': ['max', 'mean', 'sum', 'var'],
    'SK_DPD_DEF': ['max', 'mean', 'sum'],
    'LATE_PAYMENT': ['mean']
}

In [9]:
# Flag months with late payment
pos['LATE_PAYMENT'] = pos['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)
# Aggregate by SK_ID_CURR
categorical_agg = {key: ['mean'] for key in categorical_cols}
pos_agg = group(pos, 'POS_', {**POS_CASH_AGG, **categorical_agg})

In [10]:
# Sort and group by SK_ID_PREV
sort_pos = pos.sort_values(by=['SK_ID_PREV', 'MONTHS_BALANCE'])
gp = sort_pos.groupby('SK_ID_PREV')
gp.head(10)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Amortized debt,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Returned to the store,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_XNA,LATE_PAYMENT
6030662,1000001,158271,-10,12.0,12.0,0,0,1,0,0,0,0,0,0,0,0,0
8470736,1000001,158271,-9,12.0,11.0,0,0,1,0,0,0,0,0,0,0,0,0
45995,1000001,158271,-8,2.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0
4467804,1000002,101962,-54,4.0,4.0,0,0,1,0,0,0,0,0,0,0,0,0
3346102,1000002,101962,-53,6.0,3.0,0,0,1,0,0,0,0,0,0,0,0,0
2549591,1000002,101962,-52,6.0,2.0,0,0,1,0,0,0,0,0,0,0,0,0
2658581,1000002,101962,-51,6.0,1.0,0,0,1,0,0,0,0,0,0,0,0,0
2792302,1000002,101962,-50,4.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0
6584537,1000003,252457,-4,12.0,12.0,0,0,1,0,0,0,0,0,0,0,0,0
6496374,1000003,252457,-3,12.0,11.0,0,0,1,0,0,0,0,0,0,0,0,0


In [12]:
 gp['CNT_INSTALMENT'].first().head(5)

SK_ID_PREV
1000001    12.0
1000002     4.0
1000003    12.0
1000004    10.0
1000005    10.0
Name: CNT_INSTALMENT, dtype: float64

In [13]:
gp['CNT_INSTALMENT'].last().head(5)

SK_ID_PREV
1000001     2.0
1000002     4.0
1000003    12.0
1000004     7.0
1000005    10.0
Name: CNT_INSTALMENT, dtype: float64